In [1]:
import os
import json

from pathlib import Path
from collections import defaultdict, Counter
from itertools import chain, starmap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [178]:
df_15 = df_15_bu.copy()
df_15[df_15.columns[-7:-1]] = df_15[df_15.columns[-6:]]
df_15.drop("Station", axis=1, inplace=True)

In [179]:
df_15.head()

,Status,Mnet_Id,Elevation,Name,Stid,Elev_Dem,Longitude,State,Restricted,Qc_Flagged,Latitude,Timezone,Id,Period_Of_Record.Start,Period_Of_Record.End,Observations.Date_Time,Observations.Relative_Humidity_Set_1,Observations.Wind_Speed_Set_1,Observations.Wind_Gust_Set_1
0,ACTIVE,231,6388,SCE Marion Ridge,SE001,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11T20:11:00Z,2021-06-09T17:50:00Z,2021-01-15T00:00:00Z,22.64,3.33,6.72
1,ACTIVE,231,6388,SCE Marion Ridge,SE001,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11T20:11:00Z,2021-06-09T17:50:00Z,2021-01-15T00:10:00Z,24.48,4.06,10.45
2,ACTIVE,231,6388,SCE Marion Ridge,SE001,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11T20:11:00Z,2021-06-09T17:50:00Z,2021-01-15T00:20:00Z,25.66,0.78,2.34
3,ACTIVE,231,6388,SCE Marion Ridge,SE001,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11T20:11:00Z,2021-06-09T17:50:00Z,2021-01-15T00:30:00Z,25.29,1.46,4.89
4,ACTIVE,231,6388,SCE Marion Ridge,SE001,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11T20:11:00Z,2021-06-09T17:50:00Z,2021-01-15T00:40:00Z,25.61,1.72,3.43


In [182]:
dfs = [df_12, df_15, df_18]

In [183]:
df = pd.concat(dfs)

In [184]:
df["Station"] = df.Stid.str.replace("SE", "").apply(lambda i: int(i))

In [195]:
df = df.rename(columns = {"Observations.Date_Time":"Time", "Observations.Relative_Humidity_Set_1":"RelHum", "Observations.Wind_Speed_Set_1":"WindSpd", "Observations.Wind_Gust_Set_1":"GustSpd"})

In [186]:
df.reset_index(drop=True, inplace=True)

In [196]:
df.to_csv("./sce.csv", index=False)

In [162]:
len(df.index)

1013946

In [235]:
sce = pd.read_csv("sce.csv", parse_dates = ["Time", "Period_Of_Record.Start", "Period_Of_Record.End"])

In [260]:
pd.to_datetime(sce.Time).astype("<M8[ns]")

0         2021-01-12 00:00:00
1         2021-01-12 00:10:00
2         2021-01-12 00:20:00
3         2021-01-12 00:30:00
4         2021-01-12 00:40:00
                  ...        
1013941   2021-01-20 23:20:00
1013942   2021-01-20 23:30:00
1013943   2021-01-20 23:40:00
1013944   2021-01-20 23:50:00
1013945   2021-01-21 00:00:00
Name: Time, Length: 1013946, dtype: datetime64[ns]

In [215]:
sce.WindSpeed.describe(percentiles=np.arange(0,1,.01)).iloc[-10]

86%    11.86
87%    12.38
88%    12.92
89%    13.50
90%    14.13
91%    14.82
92%    15.58
93%    16.42
94%    17.37
95%    18.50
96%    19.91
97%    21.69
98%    24.18
99%    27.99
max    70.64
Name: WindSpeed, dtype: float64

In [220]:
sce["Date"] = sce.Time.dt.date
sce.rename(columns={"GustSpeed": "GustSpd", "WindSpeed": "WindSpd", "Stid":"Station"}, inplace=True)
sce.head()

,Status,Mnet_Id,Elevation,Name,Elev_Dem,Longitude,State,Restricted,Qc_Flagged,Latitude,Timezone,Id,Period_Of_Record.Start,Period_Of_Record.End,Time,RelHum,WindSpd,GustSpd,Date
0,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:00:00+00:00,30.89,2.87,6.43,2021-01-12
1,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:10:00+00:00,30.83,3.27,8.26,2021-01-12
2,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:20:00+00:00,31.16,2.16,6.06,2021-01-12
3,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:30:00+00:00,32.43,1.29,4.38,2021-01-12
4,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:40:00+00:00,32.18,1.32,3.58,2021-01-12


In [224]:
sce_15 = pd.read_csv("sce_15_18.csv")
sce_15.head()

,STATUS,MNET_ID,PERIOD_OF_RECORD,ELEVATION,NAME,STID,SENSOR_VARIABLES,ELEV_DEM,LONGITUDE,STATE,OBSERVATIONS,RESTRICTED,QC_FLAGGED,LATITUDE,TIMEZONE,ID
0,ACTIVE,231,"{'start': '2018-06-11T20:11:00Z', 'end': '2021...",6388,SCE Marion Ridge,SE001,"{'wind_speed': {'wind_speed_set_1': {}}, 'date...",6400.9,-116.73432,CA,"{'date_time': ['2021-01-15T00:00:00Z', '2021-0...",False,False,33.76594,America/Los_Angeles,70844
1,ACTIVE,231,"{'start': '2018-06-11T20:11:00Z', 'end': '2021...",5598,SCE Idyllwild,SE002,"{'wind_speed': {'wind_speed_set_1': {}}, 'date...",5587.3,-116.71663,CA,"{'date_time': ['2021-01-15T00:00:00Z', '2021-0...",False,False,33.75136,America/Los_Angeles,70845
2,ACTIVE,231,"{'start': '2018-06-11T20:11:00Z', 'end': '2021...",2750,SCE Saddle Peak,SE003,"{'wind_speed': {'wind_speed_set_1': {}}, 'date...",2732.9,-118.65901,CA,"{'date_time': ['2021-01-15T00:00:00Z', '2021-0...",False,False,34.07684,America/Los_Angeles,70846
3,ACTIVE,231,"{'start': '2018-06-14T20:32:00Z', 'end': '2021...",760,SCE Santa Anita Canyon 2,SE004,"{'wind_speed': {'wind_speed_set_1': {}}, 'date...",761.2,-118.02279,CA,"{'date_time': ['2021-01-15T00:00:00Z', '2021-0...",False,False,34.16891,America/Los_Angeles,70847
4,ACTIVE,231,"{'start': '2018-06-11T20:11:00Z', 'end': '2021...",1658,SCE Boquet Canyon,SE005,"{'wind_speed': {'wind_speed_set_1': {}}, 'date...",1663.4,-118.45637,CA,"{'date_time': ['2021-01-15T00:00:00Z', '2021-0...",False,False,34.49828,America/Los_Angeles,70848


In [237]:
sce.head()

,Unnamed: 0,Status,Mnet_Id,Elevation,Name,Elev_Dem,Longitude,State,Restricted,Qc_Flagged,Latitude,Timezone,Id,Period_Of_Record.Start,Period_Of_Record.End,Time,RelHum,WindSpd,GustSpd,Date
0,0,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:00:00+00:00,30.89,2.87,6.43,2021-01-12
1,1,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:10:00+00:00,30.83,3.27,8.26,2021-01-12
2,2,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:20:00+00:00,31.16,2.16,6.06,2021-01-12
3,3,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:30:00+00:00,32.43,1.29,4.38,2021-01-12
4,4,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:40:00+00:00,32.18,1.32,3.58,2021-01-12


In [241]:
sce_m = sce.merge(sce_15[["STID", "NAME"]], left_on="Name", right_on="NAME")
sce_m.drop(["NAME", "Unnamed: 0"], axis=1, inplace=True)
sce_m.rename(columns={"STID":"Stid"}, inplace=True)
sce_m.head()

,Status,Mnet_Id,Elevation,Name,Elev_Dem,Longitude,State,Restricted,Qc_Flagged,Latitude,Timezone,Id,Period_Of_Record.Start,Period_Of_Record.End,Time,RelHum,WindSpd,GustSpd,Date,Stid
0,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:00:00+00:00,30.89,2.87,6.43,2021-01-12,SE001
1,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:10:00+00:00,30.83,3.27,8.26,2021-01-12,SE001
2,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:20:00+00:00,31.16,2.16,6.06,2021-01-12,SE001
3,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:30:00+00:00,32.43,1.29,4.38,2021-01-12,SE001
4,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:40:00+00:00,32.18,1.32,3.58,2021-01-12,SE001


In [243]:
sce_m.rename(columns={"Stid":"Station"}, inplace=True)
sce_m.head()

,Status,Mnet_Id,Elevation,Name,Elev_Dem,Longitude,State,Restricted,Qc_Flagged,Latitude,Timezone,Id,Period_Of_Record.Start,Period_Of_Record.End,Time,RelHum,WindSpd,GustSpd,Date,Station
0,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:00:00+00:00,30.89,2.87,6.43,2021-01-12,SE001
1,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:10:00+00:00,30.83,3.27,8.26,2021-01-12,SE001
2,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:20:00+00:00,31.16,2.16,6.06,2021-01-12,SE001
3,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:30:00+00:00,32.43,1.29,4.38,2021-01-12,SE001
4,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:40:00+00:00,32.18,1.32,3.58,2021-01-12,SE001


In [244]:
sce_m.to_csv("sce.csv", index=False)

In [245]:
sce_m.head()

,Status,Mnet_Id,Elevation,Name,Elev_Dem,Longitude,State,Restricted,Qc_Flagged,Latitude,Timezone,Id,Period_Of_Record.Start,Period_Of_Record.End,Time,RelHum,WindSpd,GustSpd,Date,Station
0,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:00:00+00:00,30.89,2.87,6.43,2021-01-12,SE001
1,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:10:00+00:00,30.83,3.27,8.26,2021-01-12,SE001
2,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:20:00+00:00,31.16,2.16,6.06,2021-01-12,SE001
3,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:30:00+00:00,32.43,1.29,4.38,2021-01-12,SE001
4,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:40:00+00:00,32.18,1.32,3.58,2021-01-12,SE001
